In [ ]:
import sisl
import numpy as np
import os
import matplotlib.pyplot as plt
import netCDF4 as nc4
%matplotlib inline

# 1. Introduction #

In this tutorial we will showcase some of the features of [Inelastica](https://github.com/tfrederiksen/inelastica/), using a simple model to discuss tunneling through a CO molecule adsorbed on Cu.

On real Cu surfaces CO is known to adsorb on the top site in an upright configuration with the C atom towards the surface.

This is the general setup we will work with:

<img src="setup.png">

The 1D Cu electrodes are characterized by the lattice constant *a*. The electrodes are defined as 4-atom blocks of the chains to avoid coupling next-nearest layer coupling between the principal layers. The separation between "substrate" (left electrode) and "STM tip" (right electrode) is defined as *L*.

In the following, we will use [sisl](https://github.com/zerothi/sisl/) to construct geometries and [siesta](https://gitlab.com/siesta-project/siesta/) to relax geometries and determine the electronic Hamiltonian from DFT with finite differences.

Finally, we will use different scripts from [Inelastica](https://github.com/tfrederiksen/inelastica/) to compute core physical properties
* vibrational modes and energies
* eigenchannel scattering states
* inelastic transport characteristics (IETS spectra)



# 2. SIESTA calculations #

## Electrodes ##

To keep things as computationally light as possible, we consider the CO molecule between one-dimensional Cu chains as described above.

Let us begin with looking at the electronic structure of the electrodes.

In [ ]:
a = 2.56 # Cu lattice constant
s = 7.00 # transverse cell separation
Cu = sisl.Atom('Cu', tag='Cu.mpn')

# primitive cell of the electrode
folder = 'PRIM'
sc = sisl.SuperCell([a, s, s], nsc=[3, 1, 1])
elec = sisl.Geometry([0, 0, 0], atoms=Cu, sc=sc)
elec.write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

We can readily look at the electronic bands with sisl

In [ ]:
H = sisl.get_sile(f'{folder}/elec.TSHS').read_hamiltonian()
band = sisl.BandStructure(H, [[0., 0., 0.], [1/2, 1/2, 0]], 101, [r'\Gamma', 'X'])
eigs = band.apply.array.eigh()
lk = band.lineark()
plt.plot(lk, eigs, 'b');
plt.xlabel('Gamma-X'); 
plt.ylabel('$E-E_F$ (eV)');
plt.ylim(-3, 3);

Next we build a larger 4-atom electrode block to have interactions between nearest-neighbor cells only

In [ ]:
# transiesta cell for both electrodes
n = 4 # atoms in electrode block
folder = 'ELEC'
elec.tile(n, axis=0).write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

**Questions:**
* What are the symmetry character of the bands?
* What controls their alignment and band width relative to the Fermi energy?

## Geometry relaxation of device region ##

Next we build the central device region consisting of left electrode, CO molecule, and right electrode. Conventionally we call the geometry relaxation step to determine the device geometry *CGrun* (but this is just a name):

In [ ]:
folder = 'CGrun'
C = sisl.Atom('C', tag='C.mpn')
O = sisl.Atom('O', tag='O.mpn')
# CO molecule
sc = sisl.SuperCell([4, s, s], nsc=[3, 1, 1])
#CO = sisl.Geometry([[-0.6603, 0, 0], [0.5753, 0, 0]], atoms=[C, O], sc=sc) # 18-atom cell
CO = sisl.Geometry([[-0.6503, 0, 0], [0.5853, 0, 0]], atoms=[C, O], sc=sc) # 16-atom cell
left = elec.tile(6, axis=0)
device = left.append(CO, 0).append(left, 0)
device.write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

**Questions:**
* What is the CO bond length in this setup?
* What are the residual forces on the C, O and surface Cu atoms?

## Finite displacements run ##

To determine the dynamical properties of the system, as well as the electron-phonon couplings (EPC), we use finite displacements along the cartesian atoms of a selected set of **dynamical atoms** within the device, in our case we just consider the motion of CO (atom indices 7 and 8 in the device).

*Inelastica* comes with a script to prepare a finite-difference run *FCrun* for SIESTA based on the input parameters and geometry from the relaxation *CGrun* above:


In [ ]:
folder = 'FCrun'
#os.system(f'setupFCrun --FCfirst 7 --FClast 8 CGrun {folder}')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

**Questions:**
* Look at the SIESTA input file RUN.fdf: The *setupFCrun* script has prepended lines to the file. What do they mean?
* Which *.TSHS* files have SIESTA generated?
* What is contained in the *.FC* output file?

In order to compute EPC with a non-orthogonal LCAO basis, we need to know how the overlap matrix is changed between the configurations mapped out during the finite-difference *FCrun*.

To compute EPC *Inelastica* expects to find an overlap *OSrun* folder with that info. Again, *Inelastica* comes with a script to prepare the input files for SIESTA: 

In [ ]:
folder = 'OSrun'
#os.system(f'setupOSrun CGrun {folder}')
for i in range(1, 7):
    print(i)
    #os.system(f'cd {folder} && siesta RUN_{i}.fdf |tee RUN_{i}.out')
    

**Questions:**
* The *setupOSrun* has generated six input files (RUN_i.fdf) and structures (STRUCT_i.fdf). What are the differences between these setups?
* Why are these calculations extremely fast compared to a normal SIESTA run?

# 3. Transport calculation (TranSIESTA/TBTRANS) #

Above we determined the geometry using periodic boundary conditions. Let us stick to this geometry for now, but set up all inputs required for a TranSIESTA run in order to compute the transmission function with *tbtrans*:

In [ ]:
folder = 'TSrun'
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')
#os.system(f'cd {folder} && tbtrans RUN.fdf |tee RUN.TBT.out')

In [ ]:
tbt = sisl.get_sile(f'{folder}/device.TBT.nc')
plt.plot(tbt.E, tbt.transmission());
plt.xlabel('$E - E_F$ (eV)'); plt.ylabel('Transmission T(E)');

**Questions:**
* What is the typical tunneling probability of electrons near the Fermi level?
* Why are thee spikes in the transmission?

# 4. Inelastica #

## EigenChannels ##

Before discussing vibrations and inelastic effects, let us first understand the scattering states of our system. These can be computed with the *EigenChannels* script in *Inelastica*:

In [ ]:
folder = 'ECrun'
energy = -1.0
#os.system(f'EigenChannels -F 1 -L 14 -f TSrun/RUN.fdf -e {energy} -n 3 {folder}')

In the output *ECrun* folder you will find many files, among them the scattering states in the XSF format, ready for visualization with xcrysden:

In [ ]:
# sudo apt install xcrysden # Ubuntu/Linux install 
# xcrysden --xsf device.EC.1L_E-1.000.XSF

**Questions:**
* What is the meaning of the **-n 3** flag to *EigenChannels*?
* What are the symmetries of the scattering states at 1.0 eV below the Fermi level?
* What scattering states are available for transport at the Fermi level in this model system?

## Calculation of vibrational modes, energies, and EPC ##

We are now ready to look at the vibrational modes. Again, *Inelastica* has a script to analyze the data from the *FCrun* and *OSrun* above to 
* construct the dynamical matrix **D**, 
* symmetrize and diagonalize **D** to obtain normal coordinates and mode energies,
* collect the *.TSHS* files to compute the EPC corresponding to each normal coordinate

The calculation proceeds as follows:

In [ ]:
folder = 'PHrun'
#os.system(f'Phonons -c --FCfirst=7 --FClast=8 -F 5 -L 10 {folder}')

**Questions:**
* Look at the log file from the run. What has been computed?
* What changes if one excludes the **-c** flag from the call?
* What are the meanings of the other flags above?

The *Phonons* script writes several files, among them basically all quantities to a file in the netCDF4 file format. Let us use it to plot the vibrational energies:

In [ ]:
PH = nc4.Dataset(f'{folder}/Output.nc', 'r');
hw = PH.variables['hw'][:];
plt.plot(hw, 'o-');
plt.xlabel('Mode index');
plt.ylabel('Mode energy (eV)');

**Question**
* Why are there 6 modes?
* What is the meaning of negative mode energies?

To understand better the character of each mode one can use xcrysden to visualize the normal coordinate vectors, e.g., 

In [ ]:
# xcrysden --xsf PHrun/Output.mol.axsf
# Change from the menu "Modify -> Force Settings -> Length Factor = 2"
# Enable from the menu "Display -> Forces"

We can also develop an understanding of the different modes without a 3D visualizer, by reading the normal coordinate matrix of displacement amplitudes. The format is (mode index, atom index): 

In [ ]:
mode = 0
dx, dy, dz = PH.variables['U'][mode].T
plt.plot(dx, 'o', label='dx');
plt.plot(dy, 'o', label='dy');
plt.plot(dz, 'o', label='dz');
plt.legend()
plt.title(f'Normal mode = {mode}');
plt.xlabel('Atom index');
plt.ylabel('Displacement amplitude');

**Questions**
* Which modes are longitudinal with respect to the system axis?
* Which ones are transversal?
* Which pairs of modes are degenerate? And why?

## Calculation of IETS spectrum ##

In [ ]:
folder = 'INrun'
#os.system(f'Inelastica -p PHrun/Output.nc -f TSrun/RUN.fdf -F 5 -L 10 {folder}')

In [ ]:
IN = nc4.Dataset(f'{folder}/device.IN.nc', 'r')
V = IN.variables['V'][:]
I = IN.variables['I'][:]
dI = IN.variables['dI'][:]
ddI = IN.variables['ddI'][:]
plt.plot(V, I);
plt.plot(V, dI);
plt.plot(V, ddI);
plt.xlabel('Bias voltage (V)');
plt.ylabel('I, dI/dV, d2I/dV2');

In [ ]:
IN = nc4.Dataset(f'{folder}/device.IN.nc', 'r')
IETS = IN.variables['IETS'][:]
plt.plot(V, IETS);
plt.plot(hw, 0 * hw, 'o');
plt.xlabel('Bias voltage (V)');
plt.ylabel('IETS (1/V)');

## IETS spectrum with pi-states ## 

Now, in reality the conduction is carried by pi-states, we can mimic this with shifting the Fermi energy

In [ ]:
folder = 'INrun_shift'
energy = -1.0
Vmax = 0.050
Vrms = 0.001
os.system(f'Inelastica -p PHrun/Output.nc -f TSrun/RUN.fdf -F 5 -L 10 -e {energy} -V {Vrms} -v {Vmax} {folder}')

In [ ]:
IN = nc4.Dataset(f'{folder}/device.IN.nc', 'r')
V = IN.variables['V'][:]
IETS = IN.variables['IETS'][:]
plt.plot(V, IETS);
plt.xlabel('Bias voltage (V)');
plt.ylabel('IETS (1/V)');

# Extra questions #

## Isotope shifts ##

The vibrational modes of CO depends on the masses. By default, Inelastica assumes the atoms are the isotopes 12C and 16O. However, it is possible with Inelastica to specify different masses for the atoms: Here is an example where the masses of atom indices 6 and 7 are changed for 13C and 18O:

In [ ]:
folder = 'PHrun_isotope'
C13 = [6, 13.0]
O18 = [7, 18.0]
os.system(f'Phonons -c --FCfirst=7 --FClast=8 -F 5 -L 10 --Isotopes "[{C13}, {O18}]" {folder}')

## Geometric effects in IETS ##

* move tip chain away from symmetry axis?
* include vibrational heating?
* Construction of 2D system where k-point sampling plays a role?